# step2 : Data Exploration

In this step we are going to explore the data we have and try to understand it and meseare it so we can manipulate it. As *** once said :' if you can't meseare it, you can't prove it.'
So we are here to measure! 
let's start

Imports Python

In [1]:
#
import pandas as pd 
#
import numpy as np 
#
import seaborn as sns
#
import matplotlib.pyplot as plt
#
from matplotlib.collections import LineCollection
#
#from mpl_toolkits.basemap import Basemap
#
#from geopy.distance import distance 

sns.set()

The first step in solving any data-crunching problem is to get a fresh cup of coffee. The second is to figure out what your input looks like and what you're supposed to produce from it.

In [2]:
# cities
cities = pd.read_csv('data/cities.csv', index_col='id')
# providers
providers = pd.read_csv('data/providers.csv', index_col='id')
# stations
stations = pd.read_csv('data/stations.csv', index_col='id')
# ticket_data
#parse_dates = ['departure_ts', 'arrival_ts', 'search_ts'] # colonnes contenant des timestamps
ticket_data = pd.read_csv('data/ticket_data.csv', index_col='id')

In [3]:
datasets = [cities, providers, stations, ticket_data]
titles = ["cities", "providers",  "stations", "ticket_data"]

#To gather practical information about all datasets 
info_df = pd.DataFrame({},)
#table titles
info_df['dataset']= titles
#columns' name
info_df['cols'] = [', '.join([col for col, null in df.isnull().sum().items() ]) for df in datasets]
#number of columns
info_df['cols_no']= [df.shape[1] for df in datasets]
#number of missing data in the table
info_df['null_no']= [df.isnull().sum().sum() for df in datasets]
#number of columns having the missing data
info_df['null_cols_no']= [len([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]
#name of columns with missing data
info_df['null_cols'] = [', '.join([col for col, null in df.isnull().sum().items() if null > 0]) for df in datasets]
info_df.style.background_gradient(cmap='coolwarm')

,dataset,cols,cols_no,null_no,null_cols_no,null_cols
0,cities,"local_name, unique_name, latitude, longitude, population",5,7672,2,"unique_name, population"
1,providers,"company_id, provider_id, name, fullname, has_wifi, has_plug, has_adjustable_seats, has_bicycle, transport_type",9,26,5,"provider_id, has_wifi, has_plug, has_adjustable_seats, has_bicycle"
2,stations,"unique_name, latitude, longitude",3,0,0,
3,ticket_data,"company, o_station, d_station, departure_ts, arrival_ts, price_in_cents, search_ts, middle_stations, other_companies, o_city, d_city",11,165764,4,"o_station, d_station, middle_stations, other_companies"


we need to check if we just drop null values from the tables the percent of data we still have is enough or we have to find solutions to replace the missing data 

In [4]:
for table in datasets :
    shape=table.shape[0]
    drop=table.dropna()  #Keep the DataFrame with valid entries in the same variable
    print('After dropping missing values from, we still have {:0.1%} of the data'.format(drop.shape[0]/shape))
    

After dropping missing values from, we still have 4.6% of the data
After dropping missing values from, we still have 93.8% of the data
After dropping missing values from, we still have 100.0% of the data
After dropping missing values from, we still have 44.1% of the data


As we can see for the table cities, providers, and stations still have enough data even after droping the missing ones.
If we drop the missing data of the ticket_data's table, we stay with only 44.1% which is not 

Explore the tables

Ticket_data : Containing a ticket history

In [5]:
ticket_data.head()

,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
id,,,,,,,,,,,
6795025,8385,NaN,NaN,2017-10-13 14:00:00+00,2017-10-13 20:10:00+00,4550,2017-10-01 00:13:31.327+00,NaN,NaN,611,542
6795026,9,63.0,1044.0,2017-10-13 13:05:00+00,2017-10-14 06:55:00+00,1450,2017-10-01 00:13:35.773+00,"{149,418}",{13},611,542
6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 21:24:00+00,7400,2017-10-01 00:13:40.212+00,"{798,798,6794,6246}","{8377,8376}",611,542
6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00,2017-10-14 11:02:00+00,13500,2017-10-01 00:13:40.213+00,"{798,798,6794,6246}","{8377,8376}",611,542
6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00,2017-10-14 19:32:00+00,7710,2017-10-01 00:13:40.213+00,"{5983,5983}",{8380},611,542


we have 3 columns of timestamps, we have to parse them


In [6]:
parse_dates = ['departure_ts', 'arrival_ts', 'search_ts']
ticket_data = pd.read_csv('data/ticket_data.csv', index_col='id', parse_dates=parse_dates)

In [7]:
ticket_data.head()

,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,d_city
id,,,,,,,,,,,
6795025,8385,NaN,NaN,2017-10-13 14:00:00+00:00,2017-10-13 20:10:00+00:00,4550,2017-10-01 00:13:31.327000+00:00,NaN,NaN,611,542
6795026,9,63.0,1044.0,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,2017-10-01 00:13:35.773000+00:00,"{149,418}",{13},611,542
6795027,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 21:24:00+00:00,7400,2017-10-01 00:13:40.212000+00:00,"{798,798,6794,6246}","{8377,8376}",611,542
6795028,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 11:02:00+00:00,13500,2017-10-01 00:13:40.213000+00:00,"{798,798,6794,6246}","{8377,8376}",611,542
6795029,8381,5905.0,6495.0,2017-10-13 21:46:00+00:00,2017-10-14 19:32:00+00:00,7710,2017-10-01 00:13:40.213000+00:00,"{5983,5983}",{8380},611,542


In [8]:
ticket_data.departure_ts.describe(datetime_is_numeric=True)

count                                  74168
mean     2017-10-18 00:38:41.896855808+00:00
min                2017-09-08 06:00:00+00:00
25%                2017-10-11 13:08:15+00:00
50%                2017-10-15 19:00:48+00:00
75%                2017-10-21 12:30:08+00:00
max                2018-03-29 16:10:00+00:00
Name: departure_ts, dtype: object

we can add new columns needed to our exploration work like :
    the time of travel 
    the distance (to find min/max/mean)
We'll have the price, the time and the distance of each travel so we can compare which is better to go on train, bus or covoiturage.

In [9]:
#duration 
ticket_data['travel_duration'] = ticket_data.arrival_ts - ticket_data.departure_ts

in order to calculate the distance we need the point ( longitude , latitude) which we can get it by merging the tables.
#let's explore them first

#cities
The cities served by tictactrip (link thanks to the o_city (origin_city), d_city (destination_city) columns of ticket_data.

In [10]:
cities.head()

,local_name,unique_name,latitude,longitude,population
id,,,,,
5159,"Padua, Veneto, Italia",padua,45.406435,11.876761,209678.0
76,"Barcelona, Cataluña, España",barcelona,41.385064,2.173404,1611822.0
81,"Basel, Basel-Stadt, Schweiz",basel,47.593437,7.619812,NaN
259,"Erlangen, Bayern, Deutschland",erlangen,49.589674,11.011961,105412.0
11979,"Balș, Olt, România",balș,44.353354,24.095672,NaN


stations served by tictactrip (link via o_station, d_station from ticket_data)

In [11]:
stations.head()

,unique_name,latitude,longitude
id,,,
1,Aalen (Stuttgarter Straße),48.835296,10.092956
2,Aéroport Bordeaux-Mérignac,44.830226,-0.700883
3,Aéroport CDG,49.009900,2.559310
4,Aéroport de Berlin-Schönefeld,52.389446,13.520345
5,Aéroport de Dresden,51.123604,13.764737


Providers: info on the different providers (link via ticket_data company)

In [12]:
providers.head()

,company_id,provider_id,name,fullname,has_wifi,has_plug,has_adjustable_seats,has_bicycle,transport_type
id,,,,,,,,,
9,1,NaN,ouibus,Ouibus,True,True,True,False,bus
10,2,NaN,deinbus,Deinbus.de,False,False,False,False,bus
11,3,NaN,infobus,Infobus,False,False,False,False,bus
12,4,NaN,studentAgency,Student Agency,False,False,False,False,bus
13,5,NaN,flixbus,Flixbus,True,False,False,False,bus


Now we imported our data we can join the tables so we get to extract informations

In [13]:
# Merge des informations sur les coordonnées dans le DataFrame ticket_data
ticket= pd.merge(ticket_data, cities[['unique_name','latitude', 'longitude']], how='left', left_on=['o_city'], right_on=['id'])
ticket.rename(columns={'unique_name':'o_city_name','latitude': 'o_city_latitude', 'longitude': 'o_city_longitude'}, inplace=True)
ticket = pd.merge(ticket, cities[['unique_name','latitude', 'longitude']], how='left', left_on=['d_city'], right_on=['id'])
ticket.rename(columns={'unique_name':'d_city_name','latitude': 'd_city_latitude', 'longitude': 'd_city_longitude'}, inplace=True)

In [14]:
# Merge des informations sur les coordonnées dans le DataFrame ticket_data
ticket= pd.merge(ticket, stations[['unique_name','latitude', 'longitude']], how='left', left_on=['o_station'], right_on=['id'])
ticket.rename(columns={'unique_name':'o_station_name','latitude': 'o_station_latitude', 'longitude': 'o_station_longitude'}, inplace=True)
ticket = pd.merge(ticket, stations[['unique_name','latitude', 'longitude']], how='left', left_on=['d_station'], right_on=['id'])
ticket.rename(columns={'unique_name':'d_station_name','latitude': 'd_station_latitude', 'longitude': 'd_station_longitude'}, inplace=True)

In [15]:
ticket= pd.merge(ticket, providers[['fullname','has_adjustable_seats','has_wifi', 'transport_type']], how='left', left_on=['company'], right_on=['id'])

In [16]:
ticket

,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,...,o_station_name,o_station_latitude,o_station_longitude,d_station_name,d_station_latitude,d_station_longitude,fullname,has_adjustable_seats,has_wifi,transport_type
0,8385,NaN,NaN,2017-10-13 14:00:00+00:00,2017-10-13 20:10:00+00:00,4550,2017-10-01 00:13:31.327000+00:00,NaN,NaN,611,...,NaN,NaN,NaN,NaN,NaN,NaN,Blablacar,False,False,carpooling
1,9,63.0,1044.0,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,2017-10-01 00:13:35.773000+00:00,"{149,418}",{13},611,...,Avenue Georges Pompidou,47.896911,1.854277,Station Les Sabines,43.583932,3.860233,Ouibus,True,True,bus
2,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 21:24:00+00:00,7400,2017-10-01 00:13:40.212000+00:00,"{798,798,6794,6246}","{8377,8376}",611,...,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500,Intercités,True,False,train
3,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 11:02:00+00:00,13500,2017-10-01 00:13:40.213000+00:00,"{798,798,6794,6246}","{8377,8376}",611,...,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500,Intercités,True,False,train
4,8381,5905.0,6495.0,2017-10-13 21:46:00+00:00,2017-10-14 19:32:00+00:00,7710,2017-10-01 00:13:40.213000+00:00,"{5983,5983}",{8380},611,...,Les Aubrais,47.926740,1.906740,Montpellier,43.604900,3.880500,Intercités de Nuit,True,False,train
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74163,13,279.0,10729.0,2017-10-27 02:30:00+00:00,2017-10-27 16:00:00+00:00,2590,2017-10-19 10:35:42.943000+00:00,"{408,408}",{8371},628,...,Gare de Bercy,48.838424,2.382411,Nantes Halluchère - Route de Paris,47.212922,-1.551565,Flixbus,False,True,bus
74164,9,279.0,304.0,2017-10-27 07:00:00+00:00,2017-10-27 13:30:00+00:00,2700,2017-10-19 10:35:42.943000+00:00,"{1105,1105}",{9},628,...,Gare de Bercy,48.838424,2.382411,Gare de Haluchère-Batignolles,47.248785,-1.520898,Ouibus,True,True,bus
74165,8371,10642.0,304.0,2017-10-27 08:30:00+00:00,2017-10-27 15:15:00+00:00,3000,2017-10-19 10:35:42.943000+00:00,"{1105,1105}",{9},628,...,Gare Routière Internationale de Paris-Gallieni...,48.865601,2.414588,Gare de Haluchère-Batignolles,47.248785,-1.520898,Isilines,True,True,bus
74166,13,279.0,304.0,2017-10-27 13:25:00+00:00,2017-10-27 21:10:00+00:00,2880,2017-10-19 10:35:42.943000+00:00,"{863,863}",{13},628,...,Gare de Bercy,48.838424,2.382411,Gare de Haluchère-Batignolles,47.248785,-1.520898,Flixbus,False,True,bus


we have good informations about history of the tickets right now
it just need some adjustment

In [17]:
ticket.columns

Index(['company', 'o_station', 'd_station', 'departure_ts', 'arrival_ts',
       'price_in_cents', 'search_ts', 'middle_stations', 'other_companies',
       'o_city', 'd_city', 'travel_duration', 'o_city_name', 'o_city_latitude',
       'o_city_longitude', 'd_city_name', 'd_city_latitude',
       'd_city_longitude', 'o_station_name', 'o_station_latitude',
       'o_station_longitude', 'd_station_name', 'd_station_latitude',
       'd_station_longitude', 'fullname', 'has_adjustable_seats', 'has_wifi',
       'transport_type'],
      dtype='object')

In [18]:
ticket.shape

(74168, 28)

for more information we can classify the duration and the distance 
for that we will add first of all a column that calculate the distance

we are going to use 
class datetime.timedelta : A duration expressing the difference between two date, time, or datetime instances to microsecond resolution.[python documentation].

# !pip install haversine

Collecting haversine
  Downloading haversine-2.5.1-py2.py3-none-any.whl (6.1 kB)
Installing collected packages: haversine
Successfully installed haversine-2.5.1


In [19]:
import haversine as hs
distance = []
classe = []
n,_ = ticket.shape

for i in range(n):
    o_city = (float(ticket['o_city_latitude'].iloc[i]), float(ticket['o_city_longitude'].iloc[i]))
    d_city = (float(ticket['d_city_latitude'].iloc[i]), float(ticket['d_city_longitude'].iloc[i]))
    d = hs.haversine(o_city,d_city)

    classe_distance = ''
    if d < 200 :
        classe_distance = 'short'
    elif d < 800 :
        classe_distance = 'medium'
    elif d < 2000 :
        classe_distance = 'long'
    else :
        classe_distance = 'too_long'
    distance.append(d)
    classe.append(classe_distance)
ticket['distance (km)'] = distance
ticket['classe_distance'] = classe
    


In [20]:
ticket.head()

,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,...,o_station_longitude,d_station_name,d_station_latitude,d_station_longitude,fullname,has_adjustable_seats,has_wifi,transport_type,distance (km),classe_distance
0,8385,NaN,NaN,2017-10-13 14:00:00+00:00,2017-10-13 20:10:00+00:00,4550,2017-10-01 00:13:31.327000+00:00,NaN,NaN,611,...,NaN,NaN,NaN,NaN,Blablacar,False,False,carpooling,503.197862,medium
1,9,63.0,1044.0,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,2017-10-01 00:13:35.773000+00:00,"{149,418}",{13},611,...,1.854277,Station Les Sabines,43.583932,3.860233,Ouibus,True,True,bus,503.197862,medium
2,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 21:24:00+00:00,7400,2017-10-01 00:13:40.212000+00:00,"{798,798,6794,6246}","{8377,8376}",611,...,1.906740,Montpellier,43.604900,3.880500,Intercités,True,False,train,503.197862,medium
3,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 11:02:00+00:00,13500,2017-10-01 00:13:40.213000+00:00,"{798,798,6794,6246}","{8377,8376}",611,...,1.906740,Montpellier,43.604900,3.880500,Intercités,True,False,train,503.197862,medium
4,8381,5905.0,6495.0,2017-10-13 21:46:00+00:00,2017-10-14 19:32:00+00:00,7710,2017-10-01 00:13:40.213000+00:00,"{5983,5983}",{8380},611,...,1.906740,Montpellier,43.604900,3.880500,Intercités de Nuit,True,False,train,503.197862,medium


the classification for distance, inspire me to classify the duration of the travel too

In [21]:
from datetime import timedelta
classe=[]
for i in range(n):
    
    t = pd.to_timedelta(ticket['travel_duration'].iloc[i]).value*10**-11

    classe_time = ''
    if t < 120 :
        classe_time = 'too_short'
    elif t < 240 :
        classe_time = 'short'
    elif t < 400 :
        classe_time = 'medium'
    elif t < 600 :
        classe_time = 'long'
    else :
        classe_time = 'too_long'
    
    classe.append(classe_time)

ticket['classe_time'] = classe

In [22]:
ticket.head()

,company,o_station,d_station,departure_ts,arrival_ts,price_in_cents,search_ts,middle_stations,other_companies,o_city,...,d_station_name,d_station_latitude,d_station_longitude,fullname,has_adjustable_seats,has_wifi,transport_type,distance (km),classe_distance,classe_time
0,8385,NaN,NaN,2017-10-13 14:00:00+00:00,2017-10-13 20:10:00+00:00,4550,2017-10-01 00:13:31.327000+00:00,NaN,NaN,611,...,NaN,NaN,NaN,Blablacar,False,False,carpooling,503.197862,medium,short
1,9,63.0,1044.0,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,2017-10-01 00:13:35.773000+00:00,"{149,418}",{13},611,...,Station Les Sabines,43.583932,3.860233,Ouibus,True,True,bus,503.197862,medium,too_long
2,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 21:24:00+00:00,7400,2017-10-01 00:13:40.212000+00:00,"{798,798,6794,6246}","{8377,8376}",611,...,Montpellier,43.604900,3.880500,Intercités,True,False,train,503.197862,medium,too_long
3,8377,5905.0,6495.0,2017-10-13 13:27:00+00:00,2017-10-14 11:02:00+00:00,13500,2017-10-01 00:13:40.213000+00:00,"{798,798,6794,6246}","{8377,8376}",611,...,Montpellier,43.604900,3.880500,Intercités,True,False,train,503.197862,medium,too_long
4,8381,5905.0,6495.0,2017-10-13 21:46:00+00:00,2017-10-14 19:32:00+00:00,7710,2017-10-01 00:13:40.213000+00:00,"{5983,5983}",{8380},611,...,Montpellier,43.604900,3.880500,Intercités de Nuit,True,False,train,503.197862,medium,too_long


In [23]:
ticket=ticket [['fullname','departure_ts','arrival_ts','price_in_cents','search_ts',
                'o_city_name', 'o_city_latitude','o_city_longitude', 
                'd_city_name','d_city_latitude', 'd_city_longitude',
                'o_station_name','o_station_latitude', 'o_station_longitude',
                'd_station_name','d_station_latitude', 'd_station_longitude',
                'travel_duration','classe_time','distance (km)','classe_distance','has_adjustable_seats','has_wifi', 'transport_type'
               ]]

In [24]:
ticket.head()

,fullname,departure_ts,arrival_ts,price_in_cents,search_ts,o_city_name,o_city_latitude,o_city_longitude,d_city_name,d_city_latitude,...,d_station_name,d_station_latitude,d_station_longitude,travel_duration,classe_time,distance (km),classe_distance,has_adjustable_seats,has_wifi,transport_type
0,Blablacar,2017-10-13 14:00:00+00:00,2017-10-13 20:10:00+00:00,4550,2017-10-01 00:13:31.327000+00:00,orleans,47.907018,1.90627,montpellier,43.604452,...,NaN,NaN,NaN,0 days 06:10:00,short,503.197862,medium,False,False,carpooling
1,Ouibus,2017-10-13 13:05:00+00:00,2017-10-14 06:55:00+00:00,1450,2017-10-01 00:13:35.773000+00:00,orleans,47.907018,1.90627,montpellier,43.604452,...,Station Les Sabines,43.583932,3.860233,0 days 17:50:00,too_long,503.197862,medium,True,True,bus
2,Intercités,2017-10-13 13:27:00+00:00,2017-10-14 21:24:00+00:00,7400,2017-10-01 00:13:40.212000+00:00,orleans,47.907018,1.90627,montpellier,43.604452,...,Montpellier,43.604900,3.880500,1 days 07:57:00,too_long,503.197862,medium,True,False,train
3,Intercités,2017-10-13 13:27:00+00:00,2017-10-14 11:02:00+00:00,13500,2017-10-01 00:13:40.213000+00:00,orleans,47.907018,1.90627,montpellier,43.604452,...,Montpellier,43.604900,3.880500,0 days 21:35:00,too_long,503.197862,medium,True,False,train
4,Intercités de Nuit,2017-10-13 21:46:00+00:00,2017-10-14 19:32:00+00:00,7710,2017-10-01 00:13:40.213000+00:00,orleans,47.907018,1.90627,montpellier,43.604452,...,Montpellier,43.604900,3.880500,0 days 21:46:00,too_long,503.197862,medium,True,False,train


In [25]:
ticket.to_csv('C:/Users/EASY/Desktop/LY/DataCrunching/data/ticketsHistory.csv')